# Specification

In This python file the Design Specifications for Chip [tt10-uR-IPs](https://github.com/jonoboderbonobo/tt10-uR-IPs) is investigated. Not much thought has been put into those Specifications, but designing a Chip without a goal in mind seems boring. Therefore some rough estimates will be made. So let's make a Robot!



## Design Goals




| Design Goal                    | Value                   |
|--------------------------------|-------------------------|
| Robot size                     | 100 µm * 100 µm         |
| Chip size                      | 50 µm * 100 µm          |
| Resulting Space for Actuators  | 2 times (25 µm * 100 µm)|
| Robot speed                    | > 1 body length /second |
| Actuation Method               | Piezoelectric           |
| Locomotion Method              | Crawling                |
| Robot shape                    | Rectangular             |

<br>

| **Soft Requirements**    |Value                   |
|--------------------------|------------------------|
| Robot Coating            | PDMS                   |
| Robot Actuators          | PZT_5H                 |
| Environment Fluid        | Air                    |
| Environment Solid (Ground)| variable (SiO2)       |


The Requirements can be split into: 
Actuation which has effect on Energy Harvesting and Driver Specification
Communication speed which has effect on clock frequency
Sensor quality. So how many Sensors, what ADC resolution and sampling rate. Effects mostly size of ADCs but also Communication Speed and maybe some buffer memory

In this first Iteration I mainly focus on Actuation and minimal Communication capabilities without any Sensors. Therefore no ADCs needed and Communication is mainly used for
Position detection and getting commands from outside to change direction or change movement speed.




### Imports

In [21]:
import sympy
from sympy import symbols, Eq, sqrt, pi, latex, solveset
from IPython.display import display, Markdown, Math
import sympy.physics.units as u
from sympy.physics.units import milli, micro, centi, giga, mega, kilo, nano, pico
from sympy.physics.units import convert_to
from sympy import Rational, pi



percent = percents = Rational(1, 100)
permille = permille = Rational(1, 1000)

ten = Rational(10)

yotta = ten**24
zetta = ten**21
exa = ten**18
peta = ten**15
tera = ten**12
giga = ten**9
mega = ten**6
kilo = ten**3
deca = ten**1
deci = ten**-1
centi = ten**-2
milli = ten**-3
micro = ten**-6
nano = ten**-9
pico = ten**-12
femto = ten**-15
atto = ten**-18
zepto = ten**-21
yocto = ten**-24

rad = radian = radians = 1
deg = degree = degrees = pi/180
sr = steradian = steradians = 1
mil = angular_mil = angular_mils = 2*pi/6400


### Materials

In [22]:

# Materials
class piezo_crystal:
    def __init__(self,name, E, E_c, nu, rho, d31, d33, h, l, w):
        """
        Initialize a piezoelectric crystal material.
        :param name: Name of the material
        :param abbrev: Abbreviation of the material name
        :param E: Young's modulus 
        :param s: elastic Compliance
        :param E_c: Coercive Field 
        :param nu: Poisson's ratio
        :param rho: Density
        :param d31: Piezoelectric coefficient d31
        :param d33: Piezoelectric coefficient d33
        :param h: Height
        :param l: Length
        :param w: Width
        """
        self.name = name
        self.E = convert_to(E, [u.pascal])
        self.s = 1/self.E
        self.E_c = convert_to(E_c, [u.volt / u.meter])
        self.nu = nu
        self.rho = rho
        self.d31 = d31
        self.d33 = d33
        self.h = convert_to(h, [u.meter])
        self.l = convert_to(l, [u.meter])
        self.w = convert_to(w, [u.meter])


    
class elastic_layer:
    def __init__(self, name, E, nu, rho, h, l, w):
        """
        Initialize an elastic layer material.
        :param name: Name of the material
        :param abbrev: Abbreviation of the material name
        :param E: Young's modulus
        :param s: elastic Compliance
        :param nu: Poisson's ratio
        :param rho: Density
        :param h: Height
        :param l: Length
        :param w: Width
        """
        self.name = name
        self.E = E
        self.s = 1/self.E
        self.nu = nu
        self.rho = rho
        self.h = h
        self.l = l
        self.w = w


In [23]:
# Create objects for PZT_5A and steel
p = piezo_crystal(
    name='PZT_5A',
    E=63 * (giga * u.pascal),
    E_c= 12 * ((kilo * u.volt) / (centi * u.meter)),
    nu=0.31,
    rho=7750 * (u.kg / (u.meter**3)),
    d31=-175 * ((pico * u.meter )/ u.volt),
    d33=400 * (u.picometer / u.volt),
    h=10 *(micro* u.meter),
    l=40 * (micro* u.meter),
    w=100 *(micro* u.meter),
)

s = elastic_layer(
    name='Steel',
    E=193* (giga* u.pascal),
    nu=0.3,
    rho=7872 * (u.kg / (u.meter**3)),
    h=10 *(micro* u.meter),
    l=40 * (micro* u.meter),
    w=100 *(micro* u.meter),
)

Eq(A, 193/63)

Eq(B, 1)

Eq(C, 3936/3875)

In [25]:

#E_c = 12000 * u.volt / u.centimeter
#E_c = convert_to(E_c, [u.volt / u.micrometer])
#thickness = 20*u.micrometer
V_max = p.E_c * p.h
V_max

12*volt

### Base piezoelectric Actuation equations 

In [26]:

# Define symbols
d_33, V, d_31, l, w, h, E_33, E_11, e, K_m, f_r, N_3, N_1, delta_33, F_b, delta_31 = symbols('d_{33} V d_{31} l w h E_{33} E_{11} e K_m f_r N_3 N_1 delta_33 F_b delta_31')

# Define equations
ax_displacement = Eq(delta_33, d_33 * V)
ax_Fb = Eq(F_b, (l * w / h) * E_33 * d_33 * V)
ax_coupling = Eq(K_m, (l * w / h) * E_33)
ax_resonance = Eq(f_r, N_3 / h)


tran_displacement = Eq(delta_31, - (w / h) * d_31 * V)
tran_Fb = Eq(F_b, - l * E_11 * d_31 * V) # Blocking force. Maximum Force the Actuator can exert when its displacement is blocked. Aka the Force that can overcome fluid drag and friction
tran_coupling = Eq(K_m, (h * l / w) * E_11) # Electromechanical coupling factor. Higher is better. Max value is 1
tran_resonance = Eq(f_r, N_1 / w)

# Display equations
display(Markdown('Axial Mode'))
for eq in [ax_displacement, ax_Fb, ax_coupling, ax_resonance]:
    display(Math(latex(eq)))
display(Markdown('Transversal Mode'))
for eq in [tran_displacement, tran_Fb, tran_coupling, tran_resonance]:
    display(Math(latex(eq)))

Axial Mode

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Transversal Mode

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Szenario of Rectangular unimporph piezo actuator fixed to an elastic Layer (e.g Steel)
Metin Sitti Page 102
base Source [208]

In [33]:



E_steel, E_pzt = symbols('E_steel E_pzt')
E_steel = s.E # Young's Modulus of Steel
E_pzt = p.E # Young's Modulus of PZT

h_steel, h_pzt = symbols('h_steel h_pzt') 
h_steel = s.h # Height of Steel
h_pzt = p.h # Height of PZT

A, B, C, D= symbols('A B C D')

E_ratio = Eq(A, E_steel/E_pzt) #Ratio of Young's Modulus
h_ratio = Eq(B, h_steel/h_pzt) #Ratio of height
density_ratio = Eq(C, s.rho/p.rho) #Ratio of density

display(E_ratio, h_ratio, density_ratio)


Eq(A, 193/63)

Eq(B, 1)

Eq(C, 3936/3875)

## Circuit Estimation

## Requirements on Driver

- **Voltage Level (Displacement):** As high as possible for maximum displacement. At this scale, it is limited by the breakdown voltage of the piezo actuator rather than MOSFETs.
- **Current Level (Speed):** Low enough resistance, i.e., high enough current for the respective oscillation frequency of the actuator. The limiting factors could be:
    - Heat dissipation of MOSFETs inside the driver circuit
    - Heat dissipation of the piezo crystal
    - Frequency where actuation is efficient due to gait, friction, drag, and mechanical quality factor 